In [ ]:
import numpy as np
from sympy  import Symbol,Add,Mul,Pow,Number,symbols
from sympy import  factor,srepr,sympify,expand,sqrt,simplify
import matplotlib.pyplot as plt

In [ ]:
def create_dictionary(m):
    m_bin= bin(m)[2:][::-1]
    m_dict={}
    for i,value in enumerate(m_bin):
        m_dict[i]= int(value)
    p_dict={}
    p_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict={}
    for i in range(n):
        for j in range(i+1):
            if i != j:
                if i >= len(m_bin) :
                    pass
                elif j ==0 :
                    pass
                else :
                    if i-j >= int(np.ceil(len(m_bin)/2))-1  :
                        z_dict[(j, i)] = 0
                        
                    else:
                        z_dict[(j, i)] =  Symbol('z'+str(j)+str(i))

        z_dict[(len(m_bin)-4, len (m_bin)-1)] =0
    return m_dict , p_dict ,q_dict , z_dict

In [ ]:
def max_sum(clause):
    max=0
    if clause.func == Add :
        for t in clause.args :
            if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) 
                #print(max)
            
            if t.func == Mul :
                if  isinstance(t.args[0], Symbol)  :           ###changed  from  previous one
                    max= max+1

                if isinstance(t.args[0], Number) and t.args[0] > 0:
                    max = max + (int(t.args[0]))
                    
                if isinstance(t.args[0], Number) and t.args[0] < 0:
                    max = max + 0
                      
                     
                    
                if isinstance(t.args[0], Number) and  t.args[0] <0 :
                    
                    pass  
            if t.func == Symbol  :
                
                max=max+1
               
            #if isinstance(t ,Symbol):
             #   max=max+1
            
            '''if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) '''
              
                

            #else :
             #   max = max+1
      

    elif clause.func == Mul :
        if  isinstance(clause.args[0], Number) and  clause.args[0] >0 :
            max =max+ int(clause.args[0])
        if   isinstance(clause.args[0], Number) and  clause.args[0] <0 :
            max =max+ int(0)
            
        else :
            max=max+1

    elif clause.func == Symbol :
        max = 1
    elif isinstance(clause,Number):
        max=max+int(clause)
   
    return max

In [ ]:
def simplify_clause(clause, equation,i):
    
    clause=clause.subs(equation).expand()
    

    if clause.func == Add:
        for t in clause.args:
            if t .func == Mul and 'Pow' in  srepr(t):
                for s in t.args:
                    if s.func  == pow:
                        clause=clause.subs({s: s.args[0]})
            if t.func == pow :
                clause=clause -t +t.args[0]

            else :
                f_clause = factor(clause)
                if f_clause .func == Mul :
                    if isinstance(f_clause.args[0],Number):
                        clause = clause /f_clause.args[0]
    if i == 1:
        equation= rule_1(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 2:
        equation= rule_2(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    
    if i == 3:
        equation= rule_3(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 4:
       equation= rule_4(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
       #print(clause,equation)

    if i == 5:
       equation= rule_5(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
    
     #  print(clause,equation)
    #equation=rule_3(clause,equation)
    #clause=clause.subs(equation).expand()
    

    return  clause ,equation

In [ ]:
def create_dictionary_robust(m):
    m_bin=bin(m)[2:][::-1]
    m_dict={}
    for i,j in enumerate(m_bin):
        m_dict[i] =int (j)

    p_dict={}
    p_dict[0]=1
    
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict1={}
    z_dict2={}

    for i in range(2,len(m_bin)):
        for j in range(i-int(np.floor(np.log2(i))),i):
            if i!= j:
                z_dict1[(j, i)] =  Symbol('z'+str(j)+str(i))

    for i in range(1,len(m_bin)-1):
        for j in range(i,i+len(bin(i)[2:][::-1])+1):
            if i!= j  and j <= len(m_bin):
                z_dict2[(i, j)] =  Symbol('z'+str(i)+str(j))




    return m_dict , p_dict ,q_dict , z_dict2
                



    


In [ ]:
def create_clause1(m,p,q,z):
    '''there is another  version  of it called create_clause2 where we have taken all the z.get possible  and then we have let our rule 1 to cut all extra z '''
    clauses=[]
    clauses1=[]
    z_zero={}
    n=len(m)+int(np.ceil(len(m)/2)) -1
    for i in range(n):
        clause=0
        for j in range(i+1):
            clause+=q.get(j,0) * p.get(i-j,0)
        #clause  += - m.get(i,0)
        for j in range(i+1):
            clause += z.get((j, i), 0)
        
        if type(clause) == int:
            clause = sympify(clause)
        if clause != 0:
            max_sum1=max_sum(clause)
            if max_sum1 !=0  :
                max_carry = int(np.floor(np.log2(max_sum1)))
            else : 
                max_carry  = 0
        for j in range(len(z)):
            if j-i > max_carry:
                if z.get((i-1, j), 0) != 0:
                    z[(i-1, j)] = 0
                    z_zero['z'+str(i-1)+str(j)] = 0 
        clause  += - m.get(i,0)
        for j in range(i+1, i+5):
            if j-i <=  max_carry +1 :
               clause += - 2**(j-i) * z.get((i, j), 0) 
        
        
        if clause ==0 :
            clause = sympify(clause)
        clauses.append(clause)

    #for clause in clauses:
     #   for keys in z_zero:
     #       clause=str(clause).replace(z_zero[keys],'0')
     #   clauses1.append(clause)
            

    return clauses

In [ ]:
def create_clause2(m,p,q,z):
    '''there is another  version  of it called create_clause2 where we have taken all the z.get possible  and then we have let our rule 1 to cut all extra z values '''
    clauses=[]
    clauses1=[]
    z_zero={}
    n=len(m)+int(np.ceil(len(m)/2)) -1
    for i in range(n):
        clause=0
        for j in range(i+1):
            clause+=q.get(j,0) * p.get(i-j,0)
        #clause  += - m.get(i,0)
        for j in range(i+1):
            clause += z.get((j, i), 0)
        
        if type(clause) == int:
            clause = sympify(clause)
        if clause != 0:
            max_sum1=max_sum(clause)
            if max_sum1 !=0  :
                max_carry = int(np.ceil(np.log2(max_sum1)))
              
            else : 
                max_carry  = 0
        """for j in range(i+1):
            if j-i > max_carry:
                if z.get((i-1, j), 0) != 0:
                    z[(i-1, j)] = 0
                    z_zero['z'+str(i-1)+str(j)] = 0 """
        clause  += - m.get(i,0)
        for j in range(i+1, i+max_carry+1):
            if j-i <=  max_carry +1 :
               clause += - 2**(j-i) * z.get((i, j), 0) 
        
        
        if clause ==0 :
            clause = sympify(clause)
        clauses.append(clause)

    #for clause in clauses:
     #   for keys in z_zero:
     #       clause=str(clause).replace(z_zero[keys],'0')
     #   clauses1.append(clause)
            

    return clauses

In [ ]:
def rule_21(clause, expression):
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            expression[list(clause.free_symbols)[0] + list(clause.free_symbols)[1]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[lisit(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule21applied'))
    
    return expression

In [ ]:
def rule_31(clause, expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==1 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')  -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule31applied'))
    
    return expression

In [ ]:
def rule_11(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) >0 :
        for  t in negative:
            if   -t.args[0] > max_sum(clause):
                var= t/t.args[0]
                expression[var] =0
    if (len(expression)!= 0):
        print(('rule11applied'))
    
    #print (max_sum(clause))

    return expression

In [ ]:
def rule_51(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [ ]:
def  rule_41(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1

In [ ]:
def  rule_61(clause,expression):
    if clause.func == Symbol   and  len(list(clause.free_symbols)) == 1:
        '''if isinstance(clause.args[0],Number):
            var= clause / clause.args[0]
            sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
            rule= Symbol('x')
            if sub_clause -rule ==0 :
               expression[list(clause.free_symbols)[0]] =0'''

        print(clause.free_symbols)
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
           expression[list(clause.free_symbols)[0]] =0

    if clause.func == Mul  and   len(list(clause.free_symbols)) == 1: 
        var= clause / clause.args[0]
        sub_clause=var.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =0


In [ ]:
def rule_71(clause,expression):
    count=0
    constant =0
    negative=[]
    if clause.func==Add  :
        for t in clause.args:
            if isinstance(t, Number) and int(t) > 0:
                constant= constant + int(t)
                print(type(int(t)))
            if  isinstance(t,Symbol):
                count = count +1
                print(type(count))
            if t.func==Mul : 
                if isinstance(t.args[0],Number) and  t.args[0] < 0:

                    negative.append(- t.args[0])
                else :

                    count =count +1
                
        print(negative)


    if len(negative) > 0 and constant + count == max(negative  ) and  int(count)  > 0:
        for t in clause.args:
            if t.func == Mul  and isinstance(t.args[0] , Number) :
                var=t/t.args[0]
                expression[var] = 1
                for i in var.args:
                    expression[i] =1
            if t.func == Mul and isinstance(t.args[0], Symbol):
                for i in t.args :
                    if isinstance(i, Symbol):
                       expression[i]=1

    
            
    return expression

In [ ]:
m,p,q,z = create_dictionary_robust(3599)
p2=create_clause1(m,p,q,z)
p2


[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68 - 8*z69,
 p2 + p3*q4 + p4*q3 + q2 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 8*z811 - 2*z89,
 p4 + q4 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 1,
 -2*z1011 - 4*z1012 + z710 + z810 + z910,
 z1011 + z811 + z911 - 1,
 z1012 + z912,
 0,
 0,
 0,
 0]

In [ ]:
expression_3599={}
renew_clause_1=[]
for clauses in p2:
    
    rule_11(clauses,expression_3599)

for clauses in  p2:
    renew_clause_1.append(clauses.subs(expression_3599).expand())
renew_clause_1





rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 p1 + q1 - 1,
 p1*q1 + p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 4*z911 - 1,
 -2*z1011 + z810 + z910,
 z1011 + z911 - 1,
 0,
 0,
 0,
 0,
 0]

In [ ]:
renew_clause_2=[]
for clauses in renew_clause_1:
    
    rule_21(clauses,expression_3599)

for clauses in  renew_clause_1:
    renew_clause_2.append(clauses.subs(expression_3599).expand())
renew_clause_2



rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied


[0,
 0,
 p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
renew_clause_3=[]
for clauses in renew_clause_2:
    
    rule_11(clauses,expression_3599)

for clauses in  renew_clause_2:
    renew_clause_3.append(clauses.subs(expression_3599).expand())
renew_clause_3

rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 0,
 p2 + q2 - 1,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
renew_clause_4=[]
for clauses in renew_clause_3:
    
    rule_21(clauses,expression_3599)

for clauses in  renew_clause_3:
    renew_clause_4.append(clauses.subs(expression_3599).expand())
renew_clause_4

rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied


[0,
 0,
 0,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p3*q4 + p4*q3 + z57 + z67 - 2*z78 - 4*z79 + 1,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
expression_3599_1={}
expression_3599_1['p2'] = 0



In [ ]:
m,p,q,z= create_dictionary_robust(56153)
p3=create_clause2(m,p,q,z)
p3,m,p,q,z

([0,
  p1 + q1 - 2*z12,
  p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24,
  p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
  p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47 - 1,
  p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58,
  p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
  p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 + 2,
  p1 + p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + q1 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
  p2 + p3*q6 + p4*q5 + p5*q4 + p6*q3 + q2 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 - 1,
  p3 + p4*q6 + p5*q5 + p6*q4 + q3 - 2*z1011 - 4*z1012 - 8*z1013 + z710 + z810 + z910,
  p4 + p5*q6 + p6*q5 + q4 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 + z811 + z911 - 1,
  p5 + p6*q6 + q5 + z1012 + z1112 - 2*z1213 - 4*z1214 - 8*z1215 + z812 + z912 - 1,
  p6 + q6 + z1013 + z1113 + z1213 - 2*z1314 - 4*z1315 - 8*z1316 + z913,
  z101

In [ ]:
bin()[2:][::-1]

In [ ]:
m,p,q,z= create_dictionary_robust(291311)
p2=create_clause2(m,p,q,z)
p2

[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
 p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + p7 + q7 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 - 1,
 p1*q7 + p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + p7*q1 + p8 + q8 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
 p1*q8 + p2*q7 + p3*q6 + p4*q5 + p5*q4 + p6*q3 + p7*q2 + p8*q1 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 + 2,
 p1 + p2*q8 + p3*q7 + p4*q6 + p5*q5 + p6*q4 + p7*q3 + p8*q2 + q1 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910,
 p2 + p3*q8 + p4*q7 + p5*q6 + p6*q5 + p7*q4 + p8*q3 + q2 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 - 16*z1115 + z811 + z911,
 p3 + p4*q8 + p5*q7 + p6*q6 + p7*q5

In [ ]:
def rule_81(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) == 0 :
        for  t in clause.args:
            
            
            expression[t] =0
    

    return expression

In [ ]:
expression_1={}
renew_clause_1=[]
for clauses in p2:
    
    rule_81(clauses,expression_1)

for clauses in  p2:
    renew_clause_1.append(clauses.subs(expression_1).expand())
renew_clause_1

[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
 p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + p7 + q7 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 - 1,
 p1*q7 + p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + p7*q1 + p8 + q8 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
 p1*q8 + p2*q7 + p3*q6 + p4*q5 + p5*q4 + p6*q3 + p7*q2 + p8*q1 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 + 2,
 p1 + p2*q8 + p3*q7 + p4*q6 + p5*q5 + p6*q4 + p7*q3 + p8*q2 + q1 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910,
 p2 + p3*q8 + p4*q7 + p5*q6 + p6*q5 + p7*q4 + p8*q3 + q2 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 - 16*z1115 + z811 + z911,
 p3 + p4*q8 + p5*q7 + p6*q6 + p7*q5

In [ ]:
expression_2={}
renew_clause_2=[]
for clauses in renew_clause_1:
    
    rule_81(clauses,expression_2)

for clauses in  renew_clause_1:
    renew_clause_2.append(clauses.subs(expression_2).expand())
renew_clause_2

[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
 p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + p7 + q7 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 - 1,
 p1*q7 + p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + p7*q1 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
 p2*q7 + p3*q6 + p4*q5 + p5*q4 + p6*q3 + p7*q2 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 + 2,
 p1 + p3*q7 + p4*q6 + p5*q5 + p6*q4 + p7*q3 + q1 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910,
 p2 + p4*q7 + p5*q6 + p6*q5 + p7*q4 + q2 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 - 16*z1115 + z811 + z911,
 p3 + p5*q7 + p6*q6 + p7*q5 + q3 + z1012 + z1112 - 2*z1213 - 4*z1214 - 8*z1215 - 16*z1216 + z

In [ ]:
expression_3={}
renew_clause_3=[]
for clauses in renew_clause_2:
    
    rule_81(clauses,expression_3)

for clauses in  renew_clause_2:
    renew_clause_3.append(clauses.subs(expression_3).expand())
renew_clause_3

[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
 p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 - 1,
 p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
 p3*q6 + p4*q5 + p5*q4 + p6*q3 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 + 2,
 p1 + p4*q6 + p5*q5 + p6*q4 + q1 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910,
 p2 + p5*q6 + p6*q5 + q2 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 - 16*z1115 + z811 + z911,
 p3 + p6*q6 + q3 + z1012 + z1112 - 2*z1213 - 4*z1214 - 8*z1215 + z812 + z912 - 1,
 p4 + q4 + z1013 + z1113 + z1213 - 2*z1314 - 4*z1315 + z913 - 1,
 p5 + q5 + z1014 + z

In [ ]:
expression_4={}
renew_clause_4=[]
for clauses in renew_clause_3:
    
    rule_81(clauses,expression_4)

for clauses in  renew_clause_3:
    renew_clause_4.append(clauses.subs(expression_4).expand())
renew_clause_4

[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 8*z69 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89 - 1,
 p4*q5 + p5*q4 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 + 2,
 p1 + p5*q5 + q1 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910,
 p2 + q2 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 + z811 + z911,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 4*z1214 + z812 + z912 - 1,
 p4 + q4 + z1013 + z1113 + z1213 - 2*z1314 + z913 - 1,
 p5 + q5 + z1014 + z1114 + z1214 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
expression_5={}
renew_clause_5=[]
for clauses in renew_clause_4:
    
    rule_11(clauses,expression_5)

for clauses in  renew_clause_4:
    renew_clause_5.append(clauses.subs(expression_5).expand())
renew_clause_5

rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 p1 + q1 - 1,
 p1*q1 + p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z57 + z67 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z68 + z78 - 4*z810 - 2*z89 - 1,
 p4*q5 + p5*q4 + z79 + z89 - 2*z910 - 4*z911 + 2,
 p1 + p5*q5 + q1 - 2*z1011 - 4*z1012 + z810 + z910,
 p2 + q2 + z1011 - 2*z1112 - 4*z1113 + z911,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 4*z1214 - 1,
 p4 + q4 + z1113 + z1213 - 2*z1314 - 1,
 p5 + q5 + z1214 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
expression_6={}
renew_clause_6=[]
for clauses in renew_clause_5:
    
    rule_11(clauses,expression_6)

for clauses in  renew_clause_5:
    renew_clause_6.append(clauses.subs(expression_6).expand())
renew_clause_6

rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 p1 + q1 - 1,
 p1*q1 + p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z57 + z67 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z68 + z78 - 4*z810 - 2*z89 - 1,
 p4*q5 + p5*q4 + z79 + z89 - 2*z910 - 4*z911 + 2,
 p1 + p5*q5 + q1 - 2*z1011 - 4*z1012 + z810 + z910,
 p2 + q2 + z1011 - 2*z1112 - 4*z1113 + z911,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 1,
 p4 + q4 + z1113 + z1213 - 2*z1314 - 1,
 p5 + q5 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
expression_overall= {}
expression_7={}
renew_clause_7=[]
for clauses in renew_clause_6:
    
    rule_21(clauses,expression_7)
    rule_21(clauses,expression_overall)
print(expression_overall)

for clauses in  renew_clause_6:
    renew_clause_7.append(clauses.subs(expression_7).expand())
renew_clause_7

rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
{p1*q1: 0, p1 + q1: 1}


[0,
 0,
 p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z57 + z67 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z68 + z78 - 4*z810 - 2*z89 - 1,
 p4*q5 + p5*q4 + z79 + z89 - 2*z910 - 4*z911 + 2,
 p5*q5 - 2*z1011 - 4*z1012 + z810 + z910 + 1,
 p2 + q2 + z1011 - 2*z1112 - 4*z1113 + z911,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 1,
 p4 + q4 + z1113 + z1213 - 2*z1314 - 1,
 p5 + q5 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
expression_8={}
renew_clause_8=[]
for clauses in renew_clause_7:
    
    rule_11(clauses,expression_8)

for clauses in  renew_clause_7:
    renew_clause_8.append(clauses.subs(expression_8).expand())
renew_clause_8

rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied
rule11applied


[0,
 0,
 p2 + q2 - 1,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z57 + z67 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z68 + z78 - 4*z810 - 2*z89 - 1,
 p4*q5 + p5*q4 + z79 + z89 - 2*z910 - 4*z911 + 2,
 p5*q5 - 2*z1011 - 4*z1012 + z810 + z910 + 1,
 p2 + q2 + z1011 - 2*z1112 - 4*z1113 + z911,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 1,
 p4 + q4 + z1113 + z1213 - 2*z1314 - 1,
 p5 + q5 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:

expression_9={}
renew_clause_9=[]
for clauses in renew_clause_8:
    
    rule_21(clauses,expression_9)
    rule_21(clauses,expression_overall)
print(expression_overall)

for clauses in  renew_clause_8:
    renew_clause_9.append(clauses.subs(expression_9).expand())
renew_clause_9

rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
rule21applied
{p1*q1: 0, p1 + q1: 1, p2*q2: 0, p2 + q2: 1}


[0,
 0,
 0,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + z46 + z56 - 2*z67 - 4*z68 - 1,
 p2*q5 + p3*q4 + p4*q3 + p5*q2 + z57 + z67 - 2*z78 - 4*z79 - 1,
 p3*q5 + p4*q4 + p5*q3 + z68 + z78 - 4*z810 - 2*z89 - 1,
 p4*q5 + p5*q4 + z79 + z89 - 2*z910 - 4*z911 + 2,
 p5*q5 - 2*z1011 - 4*z1012 + z810 + z910 + 1,
 z1011 - 2*z1112 - 4*z1113 + z911 + 1,
 p3 + q3 + z1012 + z1112 - 2*z1213 - 1,
 p4 + q4 + z1113 + z1213 - 2*z1314 - 1,
 p5 + q5 + z1314 - 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
def  rule_sum_equal(clause,expression):
    '''finding out the clauses where max negative is ===  maxsum   then checking out they are 1 or 0   '''     
    
    negative = []
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) > 0 :
        
        for  t in negative :
            if -t.args[0]  == max_sum(clause):
                var = t/t.args[0]
                expression[var] =1

                
                   
                for t in negative :
                    var1 = t/t.args[0]
                    if var1 != var :
                       expression[var1] =0
                

                
                for i in clause.args:
                        if i.func == Mul and  isinstance(i.args[0],Symbol) and  isinstance(i.args[1],Symbol) :
                           expression [i.args[0]] = 1
                           expression [i.args[1]] = 1

                        
                for i in clause.args:
                        if isinstance(i,Symbol)  :
                           expression [i] = 1
                               

        
    

    return expression

In [ ]:
expression={}
for i in renew_clause_9[10].args:
                        if i.func == Mul and  isinstance(i.args[0],Symbol) and  isinstance(i.args[1],Symbol) :
                           expression [i.args[0]] = 1
                           expression [i.args[1]] = 1

expression


{p5: 1, q5: 1}

In [ ]:

for clauses in renew_clause_9:
    expression_10={}
    renew_clause_10=[]
    
    rule_sum_equal(clauses,expression_10)
    
    

    for clauses in  renew_clause_9:
        renew_clause_10.append(clauses.subs(expression_10).expand())
    print(renew_clause_10)
    
    


{}
{}
{}
{}
{}
{}
{}
{}
{z810: 1, z89: 0, p3: 1, q5: 1, p4: 1, q4: 1, p5: 1, q3: 1, z68: 1, z78: 1}
{}
{z1012: 1, z1011: 0, p5: 1, q5: 1, z810: 1, z910: 1}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [ ]:
renew_clause_4_2=[]
for clauses in renew_clause_4:
    negative = []
  
    for t in clauses.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) > 0 :
        
        for  t in negative :
             
            if   -t.args[0] == max_sum(clauses):
                var= t/t.args[0]
                renew_clause_4_1 = []
                expression_1099551473989_1 = {}
        
                rule_sum_equal(clauses,expression_1099551473989_1)

                for clauses in  renew_clause_4:
                    renew_clause_4_1.append(clauses.subs(expression_1099551473989_1).expand())
                renew_clause_4_2.append(renew_clause_4_1) 
renew_clause_4_2

In [ ]:
m,p,q,z= create_dictionary_robust(40573241562569541338036116555849141328316436958644512915075807913316843307448816685067833)
p1=create_clause2(m,p,q,z)

In [ ]:

expression_3599={}
renew_clause_1=[]
for clauses in p1:
    
    rule_21(clauses,expression_3599)

for clauses in  p1:
    renew_clause_1.append(clauses.subs(expression_3599).expand())
renew_clause_1

[0,
 p1 + q1 - 2*z12,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47 - 1,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + p5 + q5 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 - 1,
 p1*q5 + p2*q4 + p3*q3 + p4*q2 + p5*q1 + p6 + q6 + z46 + z56 - 2*z67 - 4*z68 - 8*z69,
 p1*q6 + p2*q5 + p3*q4 + p4*q3 + p5*q2 + p6*q1 + p7 + q7 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79,
 p1*q7 + p2*q6 + p3*q5 + p4*q4 + p5*q3 + p6*q2 + p7*q1 + p8 + q8 + z58 + z68 + z78 - 4*z810 - 8*z811 - 16*z812 - 2*z89,
 p1*q8 + p2*q7 + p3*q6 + p4*q5 + p5*q4 + p6*q3 + p7*q2 + p8*q1 + p9 + q9 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 16*z913 - 1,
 p1*q9 + p10 + p2*q8 + p3*q7 + p4*q6 + p5*q5 + p6*q4 + p7*q3 + p8*q2 + p9*q1 + q10 - 2*z1011 - 4*z1012 - 8*z1013 - 16*z1014 + z710 + z810 + z910 - 1,
 p1*q10 + p10*q1 + p11 + p2*q9 + p3*q8 + p4*q7 + p5*q6 + p6*q5 + p7*q4 + p8*q3 + p9*q2 + q11 + z1011 - 2*z1112 - 4*z1113 - 8*z1114 - 16*z111